In [29]:
pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import pandas as pd
import pandasql as ps

In [31]:
from google.colab import files
uploded = files.upload()

In [32]:
insurance_data = pd.read_csv('insurance_data.csv')
insurance_data

,TXN_DATE_TIME,TRANSACTION_ID,CUSTOMER_ID,POLICY_NUMBER,POLICY_EFF_DT,LOSS_DT,REPORT_DT,INSURANCE_TYPE,PREMIUM_AMOUNT,CLAIM_AMOUNT,...,CLAIM_STATUS,INCIDENT_SEVERITY,AUTHORITY_CONTACTED,ANY_INJURY,POLICE_REPORT_AVAILABLE,INCIDENT_STATE,INCIDENT_CITY,INCIDENT_HOUR_OF_THE_DAY,AGENT_ID,VENDOR_ID
0,2020-06-01 00:00:00,TXN00000001,A00003822,PLC00008468,2015-06-23,2020-05-16,2020-05-21,Health,157.13,9000,...,A,Major Loss,Police,0,1,GA,Savannah,4,AGENT00413,VNDR00556
1,2020-06-01 00:00:00,TXN00000002,A00008149,PLC00009594,2018-04-21,2020-05-13,2020-05-18,Property,141.71,26000,...,A,Total Loss,Ambulance,1,0,AL,Montgomery,0,AGENT00769,VNDR00592
2,2020-06-01 00:00:00,TXN00000003,A00003172,PLC00007969,2019-10-03,2020-05-21,2020-05-26,Property,157.24,13000,...,A,Total Loss,Police,0,1,CO,Grand Junction,19,AGENT00883,VNDR00031
3,2020-06-01 00:00:00,TXN00000004,A00007572,PLC00009292,2016-11-29,2020-05-14,2020-05-19,Health,172.87,16000,...,A,Minor Loss,Ambulance,0,0,GA,Savannah,12,AGENT00278,VNDR00075
4,2020-06-01 00:00:00,TXN00000005,A00008173,PLC00000204,2011-12-26,2020-05-17,2020-05-22,Travel,88.53,3000,...,A,Major Loss,Police,0,1,TN,Nashville,18,AGENT00636,VNDR00472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2021-06-30 00:00:00,TXN00009996,A00006029,PLC00007245,2014-04-11,2021-06-25,2021-06-26,Motor,105.75,1000,...,A,Minor Loss,Ambulance,1,1,CT,Manchester,0,AGENT00779,NaN
9996,2021-06-30 00:00:00,TXN00009997,A00003555,PLC00002590,2015-08-25,2021-06-12,2021-06-17,Life,64.59,67000,...,A,Total Loss,Police,1,0,MA,Worcester,6,AGENT00254,VNDR00598
9997,2021-06-30 00:00:00,TXN00009998,A00004230,PLC00006613,2018-10-11,2021-06-18,2021-06-23,Health,183.50,8000,...,A,Major Loss,None,1,1,OK,Oklahoma City,12,AGENT00786,VNDR00104
9998,2021-06-30 00:00:00,TXN00009999,A00006694,PLC00001781,2016-01-29,2021-06-16,2021-06-21,Motor,113.46,2000,...,A,Total Loss,Ambulance,0,1,AR,Fayetteville,6,AGENT00927,VNDR00185


In [33]:
TaskTest = ps.sqldf("""
---------------------WRITE QUERY BELOW---------------------

SELECT 
      INSURANCE_TYPE, PREMIUM_AMOUNT, 
      CASE WHEN INSURANCE_TYPE = 'Mobile' THEN (PREMIUM_AMOUNT - (PREMIUM_AMOUNT * 0.10 / 100))
           WHEN INSURANCE_TYPE = 'Travel' THEN (PREMIUM_AMOUNT - (PREMIUM_AMOUNT * 0.10 / 100))
           WHEN INSURANCE_TYPE = 'Health' THEN (PREMIUM_AMOUNT + (PREMIUM_AMOUNT * 0.7 / 100))
           WHEN INSURANCE_TYPE = 'Property' THEN (PREMIUM_AMOUNT + (PREMIUM_AMOUNT * 0.7 / 100))
           WHEN INSURANCE_TYPE = 'Life' THEN (PREMIUM_AMOUNT + (PREMIUM_AMOUNT * 0.2 / 100))
           WHEN INSURANCE_TYPE = 'Motor' THEN (PREMIUM_AMOUNT + (PREMIUM_AMOUNT * 0.2 / 100))
      ELSE 0
      END AS 'Revised_Premium_Amount'
FROM      
      insurance_data
GROUP BY 
      INSURANCE_TYPE  

---------------------WRITE QUERY ABOVE---------------------
""")

TaskTest

,INSURANCE_TYPE,PREMIUM_AMOUNT,Revised_Premium_Amount
0,Health,183.50,184.78450
1,Life,64.59,64.71918
2,Mobile,8.22,8.21178
3,Motor,113.46,113.68692
4,Property,135.33,136.27731
5,Travel,88.20,88.11180


In [28]:
TaskTest1 = ps.sqldf("""
---------------------WRITE QUERY BELOW---------------------

SELECT 
      (SUM(Revised_Premium_Amount) + SUM(PREMIUM_AMOUNT)) / SUM(PREMIUM_AMOUNT) AS OVERALL_CHANGE
FROM 
      TaskTest               

---------------------WRITE QUERY ABOVE---------------------
""")

TaskTest1

,OVERALL_CHANGE
0,2.004199
